# Import

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns


In [2]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from typing import Dict, Tuple
import torch

# Data

In [3]:
# Feature dictionary(fundamental feature)

path = r'h:\我的雲端硬碟\DEV_INTERN_ALPHA_DATA-20240709T015049Z-001\DEV_INTERN_ALPHA_DATA\pmart.parquet'
df = pd.read_parquet(path)
pmart_column_lst = df.columns.tolist()[2:]
pmart_data_dict = {}

pmart_column_lst = df.columns.tolist()[2:]
pmart_column_lst = [col for col in pmart_column_lst if col not in ['季別', '合併(Y/N)', '單季(Q)/單半年(H)', '月份', '幣別', '近一週報酬率(%)', '近一月報酬率(%)', '近一季報酬率(%)', '近一年報酬率(%)']]

for column in pmart_column_lst:
    temp_df = df[['datetime', 'symbol' , column]]
    pmart_data_dict[column] = temp_df.pivot_table(
    index='datetime',  # 行索引
    columns='symbol',  # 列名稱
    values=column,  # 數值
)
    
industry_path = r"C:\Users\User\Desktop\quant\Sector Allocation Framework under Reinforcement Learning Rotation\industry-rotation\security_properties.parquet"
property_data = pd.read_parquet(industry_path)

property_data = property_data.reset_index()
property_data = property_data[property_data['industry'].notna()]
symbol_to_industry = property_data[['symbol', 'industry']].set_index('symbol')['industry']

# 創建一個新 dictionary，用於儲存轉換後的 DataFrame
industry_data_dict = {}

for key, df in pmart_data_dict.items():
    # 重設 DataFrame 的列索引，將 symbol 映射為 industry
    df = df.rename(columns=symbol_to_industry).copy()
    
    # 基於 industry 聚合，計算平均值
    df = df.groupby(axis=1, level=0).mean()
    
    # 儲存到新的 dictionary
    industry_data_dict[key] = df

# 結果：industry_data_dict 包含的 DataFrame 是基於產業的全平均

feature_dict = pd.DataFrame()

# 合併多個df以符合RL所需格式
def reorganize_industry_data(industry_data_dict):
    """
    重組產業指標數據，將產業作為第一層，指標作為第二層
    
    Parameters:
    industry_data_dict: Dict[str, pd.DataFrame] - 指標資料字典，key為指標名稱
    
    Returns:
    pd.DataFrame - 重組後的DataFrame，MultiIndex為(產業, 指標)
    """
    # 取得所有日期作為索引
    all_dates = set()
    for df in industry_data_dict.values():
        all_dates.update(df.index)
    all_dates = sorted(list(all_dates))
    
    # 取得所有產業名稱
    industries = industry_data_dict[list(industry_data_dict.keys())[0]].columns
    
    # 創建新的DataFrame
    result_dict = {}
    
    # 重組數據
    for industry in industries:
        for indicator, df in industry_data_dict.items():
            if industry in df.columns:
                # 使用元組作為鍵，第一層是產業，第二層是指標
                result_dict[(industry, indicator)] = df[industry]
    
    # 創建最終的DataFrame
    result_df = pd.DataFrame(result_dict, index=all_dates)
    
    # 確保MultiIndex格式正確
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns)
    
    return result_df

# Fundamental feature

fundamental_lst = [
    "每股盈餘",
    "ROA(C) 稅前息前折舊前",  # 資產報酬率
    "ROE(A)-稅後",  # 股東權益報酬率
    "營業毛利率",
    "營業利益率",
    "稅後淨利率",
    "流動比率",
    "速動比率",
    "負債比率",
    "長期資金適合率(A)",
    "CFO/負債",
    "CFO/合併總損益",
    "現金流量比率",
    "營收成長率",
    "營業毛利成長率",
    "營業利益成長率"
]

fundamental_dict = {key: industry_data_dict[key] for key in fundamental_lst if key in industry_data_dict}

fundamental_feature = reorganize_industry_data(fundamental_dict)
fundamental_feature

C:\Users\User\AppData\Local\Temp\ipykernel_5832\3269719251.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_5832\3269719251.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_5832\3269719251.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_5832\3269719251.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_5832\3269719251.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.

光電業                                                           \
                每股盈餘 ROA(C) 稅前息前折舊前 ROE(A)-稅後      營業毛利率      營業利益率     稅後淨利率   
2013-05-15  0.227706       1.739083 -0.308349   9.020000  -9.252569 -7.825413   
2013-05-16  0.227706       1.739083 -0.308349   9.020000  -9.252569 -7.825413   
2013-05-17  0.227706       1.739083 -0.308349   9.020000  -9.252569 -7.825413   
2013-05-20  0.227706       1.739083 -0.308349   9.020000  -9.252569 -7.825413   
2013-05-21  0.227706       1.739083 -0.308349   9.020000  -9.252569 -7.825413   
...              ...            ...       ...        ...        ...       ...   
2024-06-13  3.347071       6.278384  2.638788  17.876162 -11.568788  1.539798   
2024-06-14  3.347071       6.278384  2.638788  17.876162 -11.568788  1.539798   
2024-06-17  3.347071       6.278384  2.638788  17.876162 -11.568788  1.539798   
2024-06-18  3.347071       6.278384  2.638788  17.876162 -11.568788  1.539798   
2024-06-19  3.347071       6.278384  2.638788  17.876162 -11.568788  1.539798   

                                                            ...    食品工業  \
                  流動比率        速動比率       負債比率   長期資金適合率(A)  ...    流動比率   
2013-05-15  193.289083  138.493853  42.542385   277.254312  ...  273.83   
2013-05-16  193.289083  138.493853  42.542385   277.254312  ...  273.83   
2013-05-17  193.289083  138.493853  42.542385   277.254312  ...  273.83   
2013-05-20  193.289083  138.493853  42.542385   277.254312  ...  273.83   
2013-05-21  193.289083  138.493853  42.542385   277.254312  ...  273.83   
...                ...         ...        ...          ...  ...     ...   
2024-06-13  354.024747  296.764545  35.277677  2662.809091  ...  261.11   
2024-06-14  354.024747  296.764545  35.277677  2662.809091  ...  261.11   
2024-06-17  354.024747  296.764545  35.277677  2662.809091  ...  261.11   
2024-06-18  354.024747  296.764545  35.277677  2662.809091  ...  261.11   
2024-06-19  354.024747  296.764545  35.277677  2662.809091  ...  261.11   

                                                                              \
                  速動比率       負債比率 長期資金適合率(A)    CFO/負債  CFO/合併總損益     現金流量比率   
2013-05-15  191.565417  39.332917  307.09125  0.181579  31.411053  13.037083   
2013-05-16  191.565417  39.332917  307.09125  0.181579  31.411053  13.037083   
2013-05-17  191.565417  39.332917  307.09125  0.181579  31.411053  13.037083   
2013-05-20  191.565417  39.332917  307.09125  0.181579  31.411053  13.037083   
2013-05-21  191.565417  39.332917  307.09125  0.181579  31.411053  13.037083   
...                ...        ...        ...       ...        ...        ...   
2024-06-13  182.222500  36.582500  255.92250  0.320000   1.542500  12.059167   
2024-06-14  182.222500  36.582500  255.92250  0.320000   1.542500  12.059167   
2024-06-17  182.222500  36.582500  255.92250  0.320000   1.542500  12.059167   
2024-06-18  182.222500  36.582500  255.92250  0.320000   1.542500  12.059167   
2024-06-19  182.222500  36.582500  255.92250  0.320000   1.542500  12.059167   

                                              
                營收成長率   營業毛利成長率      營業利益成長率  
2013-05-15   5.011667 -27.79625  1613.193333  
2013-05-16   5.011667 -27.79625  1613.193333  
2013-05-17   5.011667 -27.79625  1613.193333  
2013-05-20   5.011667 -27.79625  1613.193333  
2013-05-21   5.011667 -27.79625  1613.193333  
...               ...       ...          ...  
2024-06-13  17.015000  29.99625    34.852083  
2024-06-14  17.015000  29.99625    34.852083  
2024-06-17  17.015000  29.99625    34.852083  
2024-06-18  17.015000  29.99625    34.852083  
2024-06-19  17.015000  29.99625    34.852083  

[2716 rows x 398 columns]

In [4]:
industry_data_dict.keys()

dict_keys(['融資買進(張)', '融資賣出(張)', '融券買入(張)', '融券賣出(張)', '融資餘額(張)', '融券餘額(張)', '融資餘額(千元)', '融券餘額(千元)', '融資使用率', '融券使用率', '券資比', '融資維持率', '融券維持率', '整戶維持率', '日報酬率(%)', '開盤價', '最高價', '最低價', '收盤價', '成交量(千股)', '交易所本益比', '流通股數(千股)', '交易所淨值市價比', '現金股利率', '開盤價-除權息', '最高價-除權息', '最低價-除權息', '收盤價-除權息', '外資買賣超(千股)', '投信買賣超(千股)', '自營買賣超(千股)', '合計買賣超(千股)', '外資買進張數', '投信買進張數', '外資賣出張數', '投信賣出張數', '自營買進張數', '自營賣出張數', '外資總持股數(千股)', '投信總持股數(千股)', '自營總持股數(千股)', '外資總持股率(%)', '投信總持股率(%)', '自營總持股率(%)', '單月營收(千元)', '去年單月營收(千元)', '單月營收成長率%', '累積營收(千元)', '去年累積營收(千元)', '累積營收成長率(%)', '每股盈餘', 'ROA(C) 稅前息前折舊前', 'ROE(A)-稅後', '營業毛利率', '營業利益率', '稅後淨利率', 'CFO/負債', 'CFO/合併總損益', '現金流量比率', '每股淨值(F)-TSE公告數', '營收成長率', '營業毛利成長率', '營業利益成長率', '流動比率', '速動比率', '負債比率', '長期資金適合率(A)', '應收帳款週轉次數', '總資產週轉次數', '存貨週轉率(次)', '應收帳款及票據', '存貨', '流動資產', '採權益法之長期股權投資', '不動產廠房及設備', '商譽及無形資產合計', '非流動資產', '資產總額', '短期借款', '應付商業本票∕承兌匯票', '一年內到期長期負債', '流動負債', '應付公司債－非流動', '銀行借款－非流動', '其他長期借款－非流動', '非流動負債', '負債總額', '股本', '保留盈餘', '股東權益總額', '負債及股東權益總額',

In [5]:
# Feature lists
technical_lst = [
    "開盤價-除權息", "最高價-除權息", "最低價-除權息", "收盤價-除權息",
    "成交量(千股)", "交易所本益比", "交易所淨值市價比", "日報酬率(%)"
]

institutional_lst = [
    "外資買賣超(千股)", "投信買賣超(千股)", "自營買賣超(千股)",
    "外資總持股率(%)", "投信總持股率(%)", "自營總持股率(%)"
]

margin_lst = [
    "融資餘額(張)", "融券餘額(張)", "融資使用率", "融券使用率", "券資比"
]

# Create feature dictionaries
technical_dict = {key: industry_data_dict[key] for key in technical_lst if key in industry_data_dict}
institutional_dict = {key: industry_data_dict[key] for key in institutional_lst if key in industry_data_dict}
margin_dict = {key: industry_data_dict[key] for key in margin_lst if key in industry_data_dict}

# Add to feature_dict
price_volume_feature = reorganize_industry_data(technical_dict)
institutional_feature = reorganize_industry_data(institutional_dict)
margin_feature = reorganize_industry_data(margin_dict)

In [7]:
# return data
return_path = r'C:\Users\User\Desktop\quant\Sector Allocation Framework under Reinforcement Learning Rotation\industry-rotation\industryRL_v3\returns_data.csv'
return_data = pd.read_csv(return_path, index_col=0)
return_data 

,光電業,其他,化學工業,半導體,居家生活,建材營造,數位雲端,文化創意業,材料,汽車工業,...,資訊服務業,通信網路業,金融業,鋼鐵工業,電子商務,電子通路業,電子零組件,電機機械,電腦及週邊,食品工業
datetime,,,,,,,,,,,,,,,,,,,,,
2013-05-15,1.302608,0.642286,0.555994,0.421326,1.430424,1.199804,0.832743,0.068016,0.975402,1.106733,...,0.286967,0.635315,1.274580,0.236545,-0.3968,-0.102849,0.512790,0.240838,0.603218,0.727992
2013-05-16,1.185714,-0.036057,0.197428,0.836739,0.817339,0.699168,2.735586,0.446837,0.767758,1.139428,...,-0.042337,0.187807,2.271045,0.364575,1.1952,0.554331,0.188547,0.507773,0.340185,0.243037
2013-05-17,1.512616,0.441230,0.179203,0.155004,0.421407,0.382724,3.806300,0.589184,0.117923,0.165011,...,-0.043563,0.443001,-0.204555,0.104760,6.6929,0.521608,0.628443,0.076025,0.577626,0.914350
2013-05-20,1.834277,0.620290,0.049575,0.013956,0.202222,0.684601,1.009814,-0.270089,0.459767,0.013900,...,0.825748,0.479855,0.919307,0.421353,-4.7970,0.774603,0.669102,0.433365,0.482416,0.439271
2013-05-21,1.433981,0.840786,0.208633,1.255811,0.770324,1.245729,0.011843,-0.419305,0.657993,0.488272,...,1.186656,0.174277,-0.196455,1.180495,-0.7752,1.235131,0.763970,0.588637,0.942956,0.161783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13,0.902497,0.182257,0.061393,1.664595,0.060207,-0.052452,0.783233,0.289356,-0.222916,0.072225,...,0.321056,0.448566,0.200614,0.201419,NaN,0.759009,0.579641,0.729533,0.719946,0.022267
2024-06-14,1.033071,0.509263,-0.049464,0.548502,0.170418,1.386225,-0.216948,0.644084,0.523718,0.100047,...,0.002290,0.460051,0.164769,0.224991,NaN,0.264346,0.740764,1.100432,0.616124,0.269683
2024-06-17,0.965813,0.409280,0.470198,0.795250,0.170254,-1.090854,0.241833,1.275944,0.419556,0.200936,...,0.059705,1.048089,0.114348,0.559096,NaN,0.750651,0.612861,0.754979,0.619787,0.263377


# Def

In [8]:
class SimpleIndustryEnv(gym.Env):
    def __init__(self, features_dict: Dict[str, pd.DataFrame], returns_data: pd.DataFrame):
        super().__init__()
        
        self.features_dict = self._process_features_dict(features_dict)
        self.returns_data = returns_data
        self.industry_names = list(returns_data.columns)  # Store column names as list
        self.n_industries = len(self.industry_names)
        self.total_features = sum(df.shape[1] for df in self.features_dict.values())
        
        self.action_space = spaces.Discrete(self.n_industries)
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(self.total_features,),
            dtype=np.float32
        )
        
        self.current_step = 0
        
    def _process_features_dict(self, features_dict):
        processed = {}
        for key, df in features_dict.items():
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = ['_'.join(map(str, col)).strip() for col in df.columns]
            processed[key] = df.fillna(0)
        return processed
    
    def _get_state(self) -> np.ndarray:
        current_date = self.returns_data.index[self.current_step]
        state = []
        
        for df in self.features_dict.values():
            values = df.loc[current_date].values
            values = np.nan_to_num(values, nan=0.0)
            state.extend(values)
            
        return np.array(state, dtype=np.float32)
    
    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, dict]:
        next_returns = self.returns_data.iloc[self.current_step + 1]
        reward = float(next_returns.iloc[action])
        reward = np.nan_to_num(reward, nan=0.0)
        
        self.current_step += 1
        done = self.current_step >= len(self.returns_data) - 2
        
        return self._get_state(), reward, done, False, {}
    
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        return self._get_state(), {}

def train_and_predict(features_dict: Dict[str, pd.DataFrame], returns_data: pd.DataFrame,
                     train_size: int = 120, total_timesteps: int = 10000):
    returns_data = returns_data.fillna(0)
    
    train_features = {k: v.iloc[:train_size] for k, v in features_dict.items()}
    train_returns = returns_data.iloc[:train_size]
    test_returns = returns_data.iloc[train_size:]
    
    env = SimpleIndustryEnv(train_features, train_returns)
    
    model = PPO(
        'MlpPolicy', 
        env,
        verbose=0,
        device='cpu',
        policy_kwargs={'net_arch': [64, 64]}
    )
    
    model.learn(total_timesteps=total_timesteps)
    
    obs = env.reset()[0]
    predicted_industry_idx, _ = model.predict(obs, deterministic=True)
    
    # Calculate cumulative sum of returns for each industry
    test_returns_sum = test_returns.sum()
    
    # Convert integer index to industry name
    predicted_industry = env.industry_names[predicted_industry_idx]
    actual_best_industry = test_returns_sum.idxmax()
    
    predicted_return = float(test_returns_sum[predicted_industry])
    best_return = float(test_returns_sum.max())
    
    # Create result DataFrame
    result_df = pd.DataFrame({
        'train_start': [train_returns.index[0]],
        'train_end': [train_returns.index[-1]],
        'predict_start': [test_returns.index[0]],
        'predict_end': [test_returns.index[-1]],
        'predicted_industry': [predicted_industry],
        'actual_best_industry': [actual_best_industry],
        'predicted_return': [predicted_return],
        'best_return': [best_return],
        'prediction_accuracy': [predicted_industry == actual_best_industry],
        'return_difference': [predicted_return - best_return]
    })
    
    return result_df

# Helper function to check data quality
def check_data_quality(features_dict, returns_data):
    print("Returns Data Shape:", returns_data.shape)
    print("\nReturns Data NaN Count:\n", returns_data.isna().sum())
    
    print("\nFeatures Data:")
    for key, df in features_dict.items():
        print(f"\n{key} Shape:", df.shape)
        print(f"{key} NaN Count:\n", df.isna().sum())

In [9]:
def filter_ecommerce(features_dict, returns_data):
    # 過濾 returns_data
    non_ecommerce_returns = returns_data.drop(columns=[col for col in returns_data.columns 
                                                     if '電子商務' in col])
    
    # 過濾 features_dict
    non_ecommerce_features = {}
    for key, df in features_dict.items():
        # 移除包含電子商務的欄位
        filtered_cols = [col for col in df.columns if '電子商務' not in col]
        non_ecommerce_features[key] = df[filtered_cols]
    
    return non_ecommerce_features, non_ecommerce_returns

In [10]:
feature_dict = {}
feature_dict['fundamantal'] = fundamental_feature
feature_dict['price_volume'] = price_volume_feature
feature_dict['institutional'] = institutional_feature
feature_dict['margin'] = margin_feature


In [11]:
feature_dict, return_data = filter_ecommerce(feature_dict, return_data)

In [12]:
# 檢查數據質量
check_data_quality(feature_dict, return_data)

Returns Data Shape: (2716, 24)

Returns Data NaN Count:
 光電業      0
其他       0
化學工業     0
半導體      0
居家生活     0
建材營造     0
數位雲端     0
文化創意業    0
材料       0
汽車工業     0
油電燃氣業    0
生技醫療     0
綠能環保     0
航運業      0
觀光餐旅     0
資訊服務業    0
通信網路業    0
金融業      0
鋼鐵工業     0
電子通路業    0
電子零組件    0
電機機械     0
電腦及週邊    0
食品工業     0
dtype: int64

Features Data:

fundamantal Shape: (2716, 384)
fundamantal NaN Count:
 光電業   每股盈餘              0
      ROA(C) 稅前息前折舊前    0
      ROE(A)-稅後         0
      營業毛利率             0
      營業利益率             0
                       ..
食品工業  CFO/合併總損益         0
      現金流量比率            0
      營收成長率             0
      營業毛利成長率           0
      營業利益成長率           0
Length: 384, dtype: int64

price_volume Shape: (2716, 199)
price_volume NaN Count:
 光電業   開盤價-除權息     0
      最高價-除權息     0
      最低價-除權息     0
      收盤價-除權息     0
      成交量(千股)     0
                 ..
食品工業  收盤價-除權息     0
      成交量(千股)     0
      交易所本益比      0
      交易所淨值市價比    0
      日報酬率(%)     0
Leng

In [13]:
feature_dict['price_volume']

光電業                                                \
              開盤價-除權息    最高價-除權息    最低價-除權息    收盤價-除權息      成交量(千股)   
2013-05-15  38.338061  39.492214  37.916811  39.168114  5231.477477   
2013-05-16  39.702303  40.700423  39.284958  40.027850  7570.450450   
2013-05-17  40.242260  40.872673  39.729132  40.163731  5778.828829   
2013-05-20  40.292846  40.885087  39.957044  40.371691  5169.423423   
2013-05-21  40.790742  41.810965  40.352077  41.090105  6708.863636   
...               ...        ...        ...        ...          ...   
2024-06-13  76.152926  77.570976  74.858302  76.483032  3228.453573   
2024-06-14  76.636168  79.070593  75.887609  78.345938  3304.080573   
2024-06-17  78.514661  80.900019  77.560903  79.272297  5117.275573   
2024-06-18  80.054041  81.280249  78.479703  79.768368  4295.904043   
2024-06-19  80.301103  81.200676  78.505315  79.085641  4387.148718   

                                                  其他             ...  \
               交易所本益比  交易所淨值市價比   日報酬率(%)    開盤價-除權息    最高價-除權息  ...   
2013-05-15  26.436951  1.438829  1.302608  22.208871  22.510906  ...   
2013-05-16  13.664512  1.439459  1.185714  22.256321  22.482913  ...   
2013-05-17  13.828293  1.455045  1.512616  22.193600  22.525694  ...   
2013-05-20  13.964634  1.474505  1.834277  22.209539  22.477294  ...   
2013-05-21  14.258272  1.494182  1.433981  22.326261  22.587336  ...   
...               ...       ...       ...        ...        ...  ...   
2024-06-13  48.666711  2.199231  0.902497  56.568571  57.254729  ...   
2024-06-14  49.063421  2.226154  1.033071  56.549145  57.381395  ...   
2024-06-17  49.489868  2.244017  0.965813  56.794222  57.651378  ...   
2024-06-18  49.741974  2.246410  0.166360  57.132520  57.791078  ...   
2024-06-19  49.767105  2.228291 -0.474891  57.345349  58.097975  ...   

               電腦及週邊                 食品工業                                   \
            交易所淨值市價比   日報酬率(%)    開盤價-除權息    最高價-除權息    最低價-除權息    收盤價-除權息   
2013-05-15  1.619184  0.603218  19.883777  20.210362  19.746062  20.031823   
2013-05-16  1.572245  0.340185  20.238015  20.390046  20.007123  20.207665   
2013-05-17  1.581633  0.577626  20.361277  20.649112  20.287288  20.524538   
2013-05-20  1.594082  0.482416  20.696750  20.924169  20.575650  20.784273   
2013-05-21  1.613265  0.942956  20.931481  21.130842  20.737404  20.983454   
...              ...       ...        ...        ...        ...        ...   
2024-06-13  2.912818  0.719946  66.167333  66.588333  65.887000  66.231667   
2024-06-14  2.935636  0.616124  66.231667  66.826667  66.093333  66.505000   
2024-06-17  2.946545  0.619787  66.550667  66.867667  66.259000  66.600000   
2024-06-18  2.951818  0.118930  66.653333  66.941667  66.383333  66.676000   
2024-06-19  2.941000 -0.440368  66.716667  67.324333  66.360000  66.909000   

                                                        
                成交量(千股)     交易所本益比  交易所淨值市價比   日報酬率(%)  
2013-05-15  1394.423077  28.724231  1.875385  0.727992  
2013-05-16  1517.307692  24.910385  1.876923  0.243038  
2013-05-17  1224.269231  25.174615  1.901538  0.914350  
2013-05-20  1171.346154  25.365769  1.921923  0.439271  
2013-05-21  1012.346154  25.483462  1.936923  0.161783  
...                 ...        ...       ...       ...  
2024-06-13   537.971300  27.588519  2.244000  0.022267  
2024-06-14   507.765867  27.652963  2.252333  0.269683  
2024-06-17   474.755633  27.735926  2.256000  0.263377  
2024-06-18   694.784167  27.751481  2.258667  0.162933  
2024-06-19   795.187033  27.858889  2.270000  0.239840  

[2716 rows x 199 columns]

In [14]:
return_data

,光電業,其他,化學工業,半導體,居家生活,建材營造,數位雲端,文化創意業,材料,汽車工業,...,觀光餐旅,資訊服務業,通信網路業,金融業,鋼鐵工業,電子通路業,電子零組件,電機機械,電腦及週邊,食品工業
datetime,,,,,,,,,,,,,,,,,,,,,
2013-05-15,1.302608,0.642286,0.555994,0.421326,1.430424,1.199804,0.832743,0.068016,0.975402,1.106733,...,0.702136,0.286967,0.635315,1.274580,0.236545,-0.102849,0.512790,0.240838,0.603218,0.727992
2013-05-16,1.185714,-0.036057,0.197428,0.836739,0.817339,0.699168,2.735586,0.446837,0.767758,1.139428,...,0.188944,-0.042337,0.187807,2.271045,0.364575,0.554331,0.188547,0.507773,0.340185,0.243037
2013-05-17,1.512616,0.441230,0.179203,0.155004,0.421407,0.382724,3.806300,0.589184,0.117923,0.165011,...,0.219496,-0.043563,0.443001,-0.204555,0.104760,0.521608,0.628443,0.076025,0.577626,0.914350
2013-05-20,1.834277,0.620290,0.049575,0.013956,0.202222,0.684601,1.009814,-0.270089,0.459767,0.013900,...,0.629604,0.825748,0.479855,0.919307,0.421353,0.774603,0.669102,0.433365,0.482416,0.439271
2013-05-21,1.433981,0.840786,0.208633,1.255811,0.770324,1.245729,0.011843,-0.419305,0.657993,0.488272,...,0.478588,1.186656,0.174277,-0.196455,1.180495,1.235131,0.763970,0.588637,0.942956,0.161783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-13,0.902497,0.182257,0.061393,1.664595,0.060207,-0.052452,0.783233,0.289356,-0.222916,0.072225,...,0.305734,0.321056,0.448566,0.200614,0.201419,0.759009,0.579641,0.729533,0.719946,0.022267
2024-06-14,1.033071,0.509263,-0.049464,0.548502,0.170418,1.386225,-0.216948,0.644084,0.523718,0.100047,...,1.114505,0.002290,0.460051,0.164769,0.224991,0.264346,0.740764,1.100432,0.616124,0.269683
2024-06-17,0.965813,0.409280,0.470198,0.795250,0.170254,-1.090854,0.241833,1.275944,0.419556,0.200936,...,0.233298,0.059705,1.048089,0.114348,0.559096,0.750651,0.612861,0.754979,0.619787,0.263377


# Train and predict

In [15]:
final = pd.DataFrame()  #to save all  prediction results
feature_dict['fundamantal'] = feature_dict['fundamantal'][16:]
feature_dict['price_volume'] = feature_dict['price_volume'][16:]
feature_dict['institutional'] = feature_dict['institutional'][16:]
feature_dict['margin'] = feature_dict['margin'][16:]
return_data = return_data[16:]
check_data_quality(feature_dict, return_data)

Returns Data Shape: (2700, 24)

Returns Data NaN Count:
 光電業      0
其他       0
化學工業     0
半導體      0
居家生活     0
建材營造     0
數位雲端     0
文化創意業    0
材料       0
汽車工業     0
油電燃氣業    0
生技醫療     0
綠能環保     0
航運業      0
觀光餐旅     0
資訊服務業    0
通信網路業    0
金融業      0
鋼鐵工業     0
電子通路業    0
電子零組件    0
電機機械     0
電腦及週邊    0
食品工業     0
dtype: int64

Features Data:

fundamantal Shape: (2700, 384)
fundamantal NaN Count:
 光電業   每股盈餘              0
      ROA(C) 稅前息前折舊前    0
      ROE(A)-稅後         0
      營業毛利率             0
      營業利益率             0
                       ..
食品工業  CFO/合併總損益         0
      現金流量比率            0
      營收成長率             0
      營業毛利成長率           0
      營業利益成長率           0
Length: 384, dtype: int64

price_volume Shape: (2700, 199)
price_volume NaN Count:
 光電業   開盤價-除權息     0
      最高價-除權息     0
      最低價-除權息     0
      收盤價-除權息     0
      成交量(千股)     0
                 ..
食品工業  收盤價-除權息     0
      成交量(千股)     0
      交易所本益比      0
      交易所淨值市價比    0
      日報酬率(%)     0
Leng

In [18]:
def rolling_train_predict(features_dict, returns_data, train_size=120, predict_size=30, total_timesteps=10000):
    """
    Implement rolling window prediction where each model is trained on train_size days 
    and predicts the next predict_size days.
    
    Args:
        features_dict: Dictionary of feature DataFrames
        returns_data: Returns DataFrame
        train_size: Number of days used for training
        predict_size: Number of days to predict ahead
        total_timesteps: Training iterations for PPO
    
    Returns:
        DataFrame containing all prediction results
    """
    
    all_predictions = []
    total_rows = len(returns_data)
    
    # 計算可以滾動的次數
    n_iterations = (total_rows - train_size - predict_size) // predict_size + 1
    
    print(f"總共需要執行 {n_iterations} 次預測")
    
    for i in range(n_iterations):
        print(f"正在執行第 {i+1}/{n_iterations} 次預測...")
        
        # 計算當前視窗的索引
        start_idx = i * predict_size
        train_end = start_idx + train_size
        
        # 選取當前視窗的訓練資料和預測區間
        current_features = {k: v.iloc[start_idx:train_end] for k, v in features_dict.items()}
        current_returns = returns_data.iloc[start_idx:train_end + predict_size]
        
        # 訓練和預測
        result_df = train_and_predict(
            current_features,
            current_returns,
            train_size=train_size,
            total_timesteps=total_timesteps
        )
        
        # 添加視窗資訊
        result_df['window_number'] = i
        result_df['window_start'] = start_idx
        result_df['window_end'] = train_end
        
        all_predictions.append(result_df)
        
        print(f"第 {i+1} 次預測完成")
    
    print("所有預測執行完畢！")
    
    # 合併所有預測結果
    if all_predictions:
        final_predictions = pd.concat(all_predictions, ignore_index=True)
        return final_predictions
    else:
        return pd.DataFrame()



In [19]:
# Usage example:
rolling_predictions = rolling_train_predict(feature_dict, return_data, 
                                         train_size=120, 
                                         predict_size=7,
                                         total_timesteps=10000)

總共需要執行 368 次預測
正在執行第 1/368 次預測...
第 1 次預測完成
正在執行第 2/368 次預測...
第 2 次預測完成
正在執行第 3/368 次預測...
第 3 次預測完成
正在執行第 4/368 次預測...
第 4 次預測完成
正在執行第 5/368 次預測...
第 5 次預測完成
正在執行第 6/368 次預測...
第 6 次預測完成
正在執行第 7/368 次預測...
第 7 次預測完成
正在執行第 8/368 次預測...
第 8 次預測完成
正在執行第 9/368 次預測...
第 9 次預測完成
正在執行第 10/368 次預測...
第 10 次預測完成
正在執行第 11/368 次預測...
第 11 次預測完成
正在執行第 12/368 次預測...
第 12 次預測完成
正在執行第 13/368 次預測...
第 13 次預測完成
正在執行第 14/368 次預測...
第 14 次預測完成
正在執行第 15/368 次預測...
第 15 次預測完成
正在執行第 16/368 次預測...
第 16 次預測完成
正在執行第 17/368 次預測...
第 17 次預測完成
正在執行第 18/368 次預測...
第 18 次預測完成
正在執行第 19/368 次預測...
第 19 次預測完成
正在執行第 20/368 次預測...
第 20 次預測完成
正在執行第 21/368 次預測...
第 21 次預測完成
正在執行第 22/368 次預測...
第 22 次預測完成
正在執行第 23/368 次預測...
第 23 次預測完成
正在執行第 24/368 次預測...
第 24 次預測完成
正在執行第 25/368 次預測...
第 25 次預測完成
正在執行第 26/368 次預測...
第 26 次預測完成
正在執行第 27/368 次預測...
第 27 次預測完成
正在執行第 28/368 次預測...
第 28 次預測完成
正在執行第 29/368 次預測...
第 29 次預測完成
正在執行第 30/368 次預測...
第 30 次預測完成
正在執行第 31/368 次預測...
第 31 次預測完成
正在執行第 32/368 次預測...
第 32 次預測完成
正在執行第 33/36

In [20]:
rolling_predictions.to_csv('rolling_predictions4.csv', index=False)

In [21]:
# 計算 False 的比例
false_count = rolling_predictions["prediction_accuracy"].value_counts()[False]
total_count = rolling_predictions["prediction_accuracy"].count()
false_ratio = false_count / total_count

print(f"False 的比例為: {false_ratio:.2%}")

False 的比例為: 94.84%
